<a href="https://colab.research.google.com/github/annurfauziyyah/SMKN2PPU/blob/master/Program_Pistachio(IPYNB).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split

# Fungsi untuk memuat dataset citra dari folder
def load_images_from_folder(folder, target_size=(100, 100)):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = load_img(img_path, target_size=target_size)  # Sesuaikan dengan ukuran citra yang diinginkan
        img_array = img_to_array(img)
        images.append(img_array)
        labels.append(folder.split('/')[-1])  # Menggunakan nama folder sebagai label
    return images, labels

# Path ke folder 'Kirmizi_Pistachio' dan 'Siirt_Pistachio'
Kirmizi_Pistachio_folder = '/kaggle/input/pistachio-image-dataset/Pistachio_Image_Dataset/Pistachio_Image_Dataset/Kirmizi_Pistachio'
Siirt_Pistachio_folder = '/kaggle/input/pistachio-image-dataset/Pistachio_Image_Dataset/Pistachio_Image_Dataset/Siirt_Pistachio'

# Memuat citra dan label dari folder 'Kirmizi_Pistachio'
Kirmizi_Pistachio_images, Kirmizi_Pistachio_labels = load_images_from_folder(Kirmizi_Pistachio_folder, target_size=(100, 100))

# Memuat citra dan label dari folder 'Siirt_Pistachio'
Siirt_Pistachio_images, Siirt_Pistachio_labels = load_images_from_folder(Siirt_Pistachio_folder, target_size=(100, 100))

# Menggabungkan citra dan label dari kedua kelas
x_data = np.array(Kirmizi_Pistachio_images + Siirt_Pistachio_images)
y_labels = np.array(Kirmizi_Pistachio_labels + Siirt_Pistachio_labels)

# Normalisasi citra
x_data = x_data.astype('float32') / 255.

# Memecah dataset menjadi data latih dan data uji
x_train, x_test, y_train, y_test = train_test_split(x_data, y_labels, test_size=0.2, random_state=42)

print(x_train.shape)
print(x_test.shape)

In [ ]:
class Autoencoder(tf.keras.Model):
    def __init__(self, latent_dim):
        super(Autoencoder, self).__init__()
        self.latent_dim = latent_dim
        self.encoder = tf.keras.Sequential([
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(latent_dim, activation='relu'),
        ])
        self.decoder = tf.keras.Sequential([
            tf.keras.layers.Dense(100*100*3, activation='sigmoid'),
            tf.keras.layers.Reshape((100, 100, 3))
        ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


In [ ]:
# Fungsi untuk memuat dataset citra dari folder
def load_images_from_folder(folder, target_size=(100, 100)):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = load_img(img_path, target_size=target_size)  # Sesuaikan dengan ukuran citra yang diinginkan
        img_array = img_to_array(img)
        images.append(img_array)
        labels.append(folder.split('/')[-1])  # Menggunakan nama folder sebagai label
    return images, labels

# Path ke folder 'Kirmizi_Pistachio' dan 'Siirt_Pistachio'
Kirmizi_Pistachio_folder = '/kaggle/input/pistachio-image-dataset/Pistachio_Image_Dataset/Pistachio_Image_Dataset/Kirmizi_Pistachio'
Siirt_Pistachio_folder = '/kaggle/input/pistachio-image-dataset/Pistachio_Image_Dataset/Pistachio_Image_Dataset/Siirt_Pistachio'

# Memuat citra dan label dari folder 'Kirmizi_Pistachio'
Kirmizi_Pistachio_images, Kirmizi_Pistachio_labels = load_images_from_folder(Kirmizi_Pistachio_folder, target_size=(100, 100))

# Memuat citra dan label dari folder 'Siirt_Pistachio'
Siirt_Pistachio_images, Siirt_Pistachio_labels = load_images_from_folder(Siirt_Pistachio_folder, target_size=(100, 100))

# Menggabungkan citra dan label dari kedua kelas
x_data = np.array(Kirmizi_Pistachio_images + Siirt_Pistachio_images)

# Normalisasi citra
x_data = x_data.astype('float32') / 255.

# Memecah dataset menjadi data latih dan data uji
x_train, x_test = train_test_split(x_data, test_size=0.2, random_state=42)

# Inisialisasi model autoencoder
latent_dim = 64
autoencoder = Autoencoder(latent_dim)

# Kompilasi model
autoencoder.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError())

# Melatih model
autoencoder.fit(x_train, x_train,
                epochs=10,
                shuffle=True,
                validation_data=(x_test, x_test))

In [ ]:
# Menghasilkan citra terenkripsi dan citra rekonstruksi
encoded_imgs = autoencoder.encoder(x_test).numpy()
decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()

In [ ]:
import matplotlib.pyplot as plt

n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
    # Tampilkan citra asli
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i])
    plt.title("Original")
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Tampilkan citra rekonstruksi
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i])
    plt.title("Reconstructed")
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

In [ ]:
# Menggabungkan citra dari kedua kelas (tanpa label)
x_data = np.array(Kirmizi_Pistachio_images + Siirt_Pistachio_images)

# Normalisasi citra
x_data = x_data.astype('float32') / 255.

# Memecah dataset menjadi data latih dan data uji
x_train, x_test = train_test_split(x_data, test_size=0.2, random_state=42)

# Menambahkan dimensi tambahan untuk channel
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

print(x_train.shape)

In [ ]:
# Menambahkan noise acak
noise_factor = 0.2
x_train_noisy = x_train + noise_factor * tf.random.normal(shape=x_train.shape)
x_test_noisy = x_test + noise_factor * tf.random.normal(shape=x_test.shape)

# Memastikan nilai piksel tetap di dalam rentang 0-1
x_train_noisy = tf.clip_by_value(x_train_noisy, clip_value_min=0., clip_value_max=1.)
x_test_noisy = tf.clip_by_value(x_test_noisy, clip_value_min=0., clip_value_max=1.)

In [ ]:
import matplotlib.pyplot as plt

n = 10
plt.figure(figsize=(20, 2))
for i in range(n):
    ax = plt.subplot(1, n, i + 1)
    plt.title("Original + Noise")
    plt.imshow(tf.squeeze(x_test_noisy[i]))
    plt.gray()
    plt.axis('off')
plt.show()

In [ ]:
from tensorflow.keras import layers, Model

# Define the Denoise class
class Denoise(Model):
    def __init__(self):
        super(Denoise, self).__init__()
        self.encoder = tf.keras.Sequential([
            layers.Input(shape=(100, 100, 3)),  # Update with the size of your images
            layers.Conv2D(16, (3, 3), activation='relu', padding='same', strides=2),
            layers.Conv2D(8, (3, 3), activation='relu', padding='same', strides=2)])

        self.decoder = tf.keras.Sequential([
            layers.Conv2DTranspose(8, kernel_size=3, strides=2, activation='relu', padding='same'),
            layers.Conv2DTranspose(16, kernel_size=3, strides=2, activation='relu', padding='same'),
            layers.Conv2D(3, kernel_size=(3, 3), activation='sigmoid', padding='same')])  # Update with the number of color channels in your dataset

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# Initialize the autoencoder model
autoencoder = Denoise()

# Compile the autoencoder model
autoencoder.compile(optimizer='adam', loss='mse')

# Train the autoencoder model
autoencoder.fit(x_train_noisy, x_train,
                epochs=10,
                shuffle=True,
                validation_data=(x_test_noisy, x_test))

In [ ]:
autoencoder.encoder.summary()

In [ ]:
autoencoder.decoder.summary()

In [ ]:
import matplotlib.pyplot as plt

# Menghitung gambar denoise dari gambar bising
encoded_imgs = autoencoder.encoder(x_test_noisy).numpy()
decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()

In [ ]:
# Plot gambar bising dan gambar denoise
n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
    # Tampilkan gambar asli + noise
    ax = plt.subplot(2, n, i + 1)
    plt.title("Original + Noise")
    plt.imshow(tf.squeeze(x_test_noisy[i]))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Tampilkan gambar denoise
    bx = plt.subplot(2, n, i + n + 1)
    plt.title("Reconstructed")
    plt.imshow(tf.squeeze(decoded_imgs[i]))
    plt.gray()
    bx.get_xaxis().set_visible(False)
    bx.get_yaxis().set_visible(False)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Fungsi untuk menampilkan gambar
def plot_images(original, noisy, denoised, title):
    plt.figure(figsize=(20, 4))
    for i in range(5):
        # Menampilkan gambar asli + noise
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(tf.squeeze(original[i]), cmap='gray')
        plt.title("Original + Noise")
        plt.axis("off")

        # Menampilkan gambar asli
        ax = plt.subplot(2, n, i + n + 1)
        plt.imshow(tf.squeeze(denoised[i]), cmap='gray')
        plt.title("Reconstructed")
        plt.axis("off")

    plt.suptitle(title)
    plt.show()

# Noise factors yang akan diuji
noise_factors = [0.1, 0.2, 0.3, 0.4, 0.5]

# Fungsi untuk menambahkan Gaussian noise ke gambar
def add_noise(images, noise_factor):
    noisy_images = images + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=images.shape)
    noisy_images = np.clip(noisy_images, 0., 1.)  # Clip nilai pixel ke rentang [0, 1]
    return noisy_images

# Menampilkan gambar untuk setiap noise factor
for noise_factor in noise_factors:
    # Menambahkan noise ke data uji
    x_test_noisy = add_noise(x_test, noise_factor)

    # Mendenoise data uji menggunakan model autoencoder
    decoded_imgs = autoencoder.predict(x_test_noisy)

    # Menampilkan gambar
    plot_images(x_test, x_test_noisy, decoded_imgs, f"Noise Factor: {noise_factor}")

In [ ]:
# Menambahkan noise ke data uji untuk satu faktor noise
noise_factor = 0.2  # Misalnya, gunakan satu faktor noise saja untuk memeriksa dimensi
x_test_noisy = add_noise(x_test, noise_factor)

# Memeriksa dimensi x_test_noisy
print("Dimensi x_test_noisy:", x_test_noisy.shape)

In [ ]:
# Menampilkan ringkasan model autoencoder
print("Ringkasan model autoencoder:")
autoencoder.summary()

In [ ]:
import numpy as np
import tensorflow as tf
from skimage.metrics import peak_signal_noise_ratio

# Fungsi untuk menghitung MSE
def calculate_mse(y_true, y_pred):
    return np.mean(np.square(y_true - y_pred))

# Fungsi untuk menghitung PSNR
def calculate_psnr(y_true, y_pred):
    y_true = tf.squeeze(y_true).numpy()  # Konversi ke NumPy array
    y_pred = tf.squeeze(y_pred).numpy()  # Konversi ke NumPy array
    return peak_signal_noise_ratio(y_true, y_pred)

# Menghitung dan menampilkan MSE, MAE, dan PSNR untuk setiap noise factor
for noise_factor in noise_factors:
    # Menambahkan noise ke data uji
    x_test_noisy = add_noise(x_test, noise_factor)

    # Mendenoise data uji menggunakan model autoencoder
    decoded_imgs = autoencoder.predict(x_test_noisy)

    # Pengecekan dimensi sebelum menghitung MSE
    if x_test.ndim == decoded_imgs.ndim:
        mse = calculate_mse(x_test, decoded_imgs)
    else:
        print("Dimensi dari x_test dan decoded_imgs tidak cocok untuk menghitung MSE.")

    # Menghitung PSNR
    psnr = calculate_psnr(x_test, decoded_imgs)

    # Menampilkan hasil
    print(f"Noise Factor: {noise_factor}")
    if 'mse' in locals():
        print(f"MSE: {mse}")
    else:
        print("MSE tidak dapat dihitung.")
    print(f"PSNR: {psnr}")
    print()


In [ ]:
import numpy as np
import tensorflow as tf

# Function to calculate Mean Squared Error (MSE)
def calculate_mse(original, denoised):
    mse = np.mean(np.square(original - denoised))
    return mse

# Function to calculate Mean Absolute Error (MAE)
def calculate_mae(original, denoised):
    mae = np.mean(np.abs(original - denoised))
    return mae

# Function to calculate Peak Signal-to-Noise Ratio (PSNR)
def calculate_psnr(original, denoised):
    mse = calculate_mse(original, denoised)
    max_pixel = 1.0
    psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
    return psnr

# Calculate and display MSE, MAE, and PSNR for each noise factor
for noise_factor in noise_factors:
    # Add noise to test data
    x_test_noisy = add_noise(x_test, noise_factor)

    # Denoise test data using autoencoder model
    decoded_imgs = autoencoder.predict(x_test_noisy)

    # Check dimensions before calculating MSE
    assert x_test.shape == decoded_imgs.shape, "Dimensions mismatch between original and denoised images"

    # Calculate MSE
    mse = calculate_mse(x_test, decoded_imgs)

    # Calculate MAE
    mae = calculate_mae(x_test, decoded_imgs)

    # Calculate PSNR
    psnr = calculate_psnr(x_test, decoded_imgs)

    # Display results
    print(f"Noise Factor: {noise_factor}")
    print(f"MSE: {mse}")
    print(f"MAE: {mae}")
    print(f"PSNR: {psnr}")
    print("-" * 30)

In [ ]:
import numpy as np
import tensorflow as tf
from skimage.metrics import peak_signal_noise_ratio
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Fungsi untuk menambahkan noise ke data uji
def add_noise(data, noise_factor):
    noisy_data = data + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=data.shape)
    noisy_data = np.clip(noisy_data, 0., 1.) # Memastikan nilai berada dalam rentang [0, 1]
    return noisy_data

# Fungsi untuk menghitung MSE
def calculate_mse(y_true, y_pred):
    return np.mean(np.square(y_true - y_pred))

# Fungsi untuk menghitung MAE
def calculate_mae(y_true, y_pred):
    return mean_absolute_error(y_true.flatten(), y_pred.flatten())

# Fungsi untuk menghitung PSNR
def calculate_psnr(y_true, y_pred):
    y_true = tf.squeeze(y_true).numpy() # Konversi ke NumPy array
    y_pred = tf.squeeze(y_pred).numpy() # Konversi ke NumPy array
    return peak_signal_noise_ratio(y_true, y_pred)

# Noise factors yang ingin diuji
noise_factors = [0.1, 0.2, 0.3, 0.4, 0.5]

# Menghitung dan menampilkan MSE, MAE, dan PSNR untuk setiap noise factor
for noise_factor in noise_factors:
    # Menambahkan noise ke data uji
    x_test_noisy = add_noise(x_test, noise_factor)

    # Mendenoise data uji menggunakan model autoencoder
    decoded_imgs = autoencoder.predict(x_test_noisy)

    # Pengecekan dimensi sebelum menghitung MSE
    if x_test.ndim == decoded_imgs.ndim:
        mse = calculate_mse(x_test, decoded_imgs)
        mae = calculate_mae(x_test, decoded_imgs)
    else:
        print("Dimensi dari x_test dan decoded_imgs tidak cocok untuk menghitung MSE.")

    # Menghitung PSNR
    psnr = calculate_psnr(x_test, decoded_imgs)

    # Menampilkan hasil
    print(f"Noise Factor: {noise_factor}")
    if 'mse' in locals():
        print(f"MSE: {mse}")
        print(f"MAE: {mae}")
    else:
        print("MSE tidak dapat dihitung.")
    print(f"PSNR: {psnr}")
    print()

In [ ]:
from sklearn.metrics import mean_squared_error

# Menghitung MSE untuk setiap pasang gambar
mse_values = []
for i in range(len(x_test_noisy)):
    # Mengonversi EagerTensor menjadi array NumPy dan meratakan gambar
    noisy_image_flat = x_test_noisy[i].numpy().flatten()
    reconstructed_image_flat = reconstructed_noisy_images[i].flatten()
    # Menghitung MSE
    mse = mean_squared_error(noisy_image_flat, reconstructed_image_flat)
    mse_values.append(mse)

# Menghitung rata-rata MSE
avg_mse = sum(mse_values) / len(mse_values)
print("Average MSE:", avg_mse)

In [ ]:
import numpy as np
import tensorflow as tf
from skimage.metrics import peak_signal_noise_ratio

# Fungsi untuk menghitung MSE
def calculate_mse(y_true, y_pred):
    return np.mean(np.square(y_true - y_pred))

# Fungsi untuk menghitung PSNR
def calculate_psnr(y_true, y_pred):
    y_true = tf.squeeze(y_true).numpy()  # Konversi ke NumPy array
    y_pred = tf.squeeze(y_pred).numpy()  # Konversi ke NumPy array
    return peak_signal_noise_ratio(y_true, y_pred)

# Menghitung dan menampilkan MSE, MAE, dan PSNR untuk setiap noise factor
for noise_factor in noise_factors:
    # Menambahkan noise ke data uji
    x_test_noisy = add_noise(x_test, noise_factor)

    # Mendenoise data uji menggunakan model autoencoder
    decoded_imgs = autoencoder.predict(x_test_noisy)

    # Pengecekan dimensi sebelum menghitung MSE
    if x_test.ndim == decoded_imgs.ndim:
        mse = calculate_mse(x_test, decoded_imgs)
    else:
        print("Dimensi dari x_test dan decoded_imgs tidak cocok untuk menghitung MSE.")

    # Menghitung PSNR
    psnr = calculate_psnr(x_test, decoded_imgs)

    # Menampilkan hasil
    print(f"Noise Factor: {noise_factor}")
    if 'mse' in locals():
        print(f"MSE: {mse}")
    else:
        print("MSE tidak dapat dihitung.")
    print(f"PSNR: {psnr}")
    print()


In [ ]:
import numpy as np
import tensorflow as tf
from skimage.metrics import peak_signal_noise_ratio

# Fungsi untuk menghitung MSE
def calculate_mse(y_true, y_pred):
    return np.mean(np.square(y_true - y_pred))

# Fungsi untuk menghitung PSNR
def calculate_psnr(y_true, y_pred):
    y_true = tf.squeeze(y_true).numpy()  # Konversi ke NumPy array
    y_pred = tf.squeeze(y_pred).numpy()  # Konversi ke NumPy array
    return peak_signal_noise_ratio(y_true, y_pred)

# Menghitung dan menampilkan MSE, MAE, dan PSNR untuk setiap noise factor
for noise_factor in noise_factors:
    # Menambahkan noise ke data uji
    x_test_noisy = add_noise(x_test, noise_factor)

    # Mendenoise data uji menggunakan model autoencoder
    decoded_imgs = autoencoder.predict(x_test_noisy)

    # Pengecekan dimensi sebelum menghitung MSE
    if x_test.ndim == decoded_imgs.ndim:
        mse = calculate_mse(x_test, decoded_imgs)
    else:
        print("Dimensi dari x_test dan decoded_imgs tidak cocok untuk menghitung MSE.")

    # Menghitung PSNR
    psnr = calculate_psnr(x_test, decoded_imgs)

    # Menampilkan hasil
    print(f"Noise Factor: {noise_factor}")
    if 'mse' in locals():
        print(f"MSE: {mse}"
        print(f"MAE: {mae}")
        print(f"PSNR: {psnr}")
        print()


In [ ]:
import numpy as np
import tensorflow as tf
from skimage.metrics import peak_signal_noise_ratio
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Fungsi untuk menghitung PSNR
def calculate_psnr(y_true, y_pred):
    y_true = tf.squeeze(y_true).numpy()  # Konversi ke NumPy array
    y_pred = tf.squeeze(y_pred).numpy()  # Konversi ke NumPy array
    return peak_signal_noise_ratio(y_true, y_pred)

# Fungsi untuk Menghitung MSE
def calculate_mse(y_true, y_pred):
    return mean_squared_error(y_true, y_pred)

# Fungsi untuk Menghitung MAE
def calculate_mae(y_true, y_pred):
    return mean_absolute_error(y_true, y_pred)

# Noise factors yang ingin diuji
noise_factors = [0.1, 0.2, 0.3, 0.4, 0.5]

# Mendenoise data uji menggunakan model autoencoder
def denoise_data(autoencoder, x_test_noisy):
    return autoencoder.predict(x_test_noisy)

# Data uji (contoh)
x_test = np.random.rand(1718, 100, 3)  # Contoh data uji
x_test_noisy = add_noise(x_test, noise_factor)  # Contoh data uji yang bising

# Menghitung dan menampilkan MSE, MAE, dan PSNR untuk setiap noise factor
for noise_factor in noise_factors:
    # Menambahkan noise ke data uji
    x_test_noisy = add_noise(x_test, noise_factor)

    # Mendenoise data uji menggunakan model autoencoder
    decoded_imgs = denoise_data(autoencoder, x_test_noisy)

    # Menghitung MSE, MAE, dan PSNR
    mse = calculate_mse(x_test, decoded_imgs)
    mae = calculate_mae(x_test, decoded_imgs)
    psnr = calculate_psnr(x_test, decoded_imgs)

    # Menampilkan hasil
    print(f"Noise Factor: {noise_factor}")
    print(f"MSE: {mse}")
    print(f"MAE: {mae}")
    print(f"PSNR: {psnr}")
    print()


In [ ]:
import matplotlib.pyplot as plt

# Fungsi untuk menampilkan gambar
def plot_images(original, noisy, denoised, title):
    plt.figure(figsize=(10, 5))
    for i in range(5):
        # Menampilkan gambar asli + noise
        ax = plt.subplot(3, 5, i + 1)
        plt.imshow(tf.squeeze(original[i]), cmap='gray')
        plt.title("Original + Noise")
        plt.axis("off")

        # Menampilkan gambar rekonstruksi
        ax = plt.subplot(3, 5, i + 6)
        plt.imshow(tf.squeeze(noisy[i]), cmap='gray')
        plt.title("Noisy Image")
        plt.axis("off")

        # Menampilkan gambar asli
        ax = plt.subplot(3, 5, i + 11)
        plt.imshow(tf.squeeze(denoised[i]), cmap='gray')
        plt.title("Reconstructed")
        plt.axis("off")

    plt.suptitle(title)
    plt.show()

# Noise factors yang akan diuji
noise_factors = [0.1, 0.2, 0.3, 0.4, 0.5]

# Menampilkan gambar untuk setiap noise factor
for noise_factor in noise_factors:
    # Menambahkan noise ke data uji
    x_test_noisy = add_noise(x_test, noise_factor)

    # Mendenoise data uji menggunakan model autoencoder
    decoded_imgs = autoencoder.predict(x_test_noisy)

    # Menampilkan gambar
    plot_images(x_test, x_test_noisy, decoded_imgs, f"Noise Factor: {noise_factor}")